In [1]:
# import ipynb
# import torch
# %run data_handler.ipynb
# #%run train_model.ipynb

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# data, description = load_subject(1)
# data, description = get_innerspeech(data, description)
# labels = extract_labels(description)
# print("1")
# data_interval = extract_action_interval(data, hz = 254)
# print("2")

# train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data_interval, labels)

# train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
# split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

1
2
Split info:
[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


In [ ]:
#train_data[0].shape

torch.Size([1, 128, 4609])

# Shallow CNN
Implementation of the shallow CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class ShallowCNN(nn.Module):
    def __init__(self, hz, interval, dropout = 0.5):
        super(ShallowCNN, self).__init__()
        first_channels = 40
        

        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 25), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = 40, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm = nn.BatchNorm2d(40, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Mean pooling
        self.meanpool = nn.AvgPool2d(kernel_size = (1,75), stride = (1,15)) # This 15 might be a 75 :(

        # Classifier
        if interval == "action":
            if hz == 254:
                self.classifier = nn.Linear(1440,4, bias = False)
            elif hz == 1024:
                self.classifier = nn.Linear(6600,4, bias = False)
        elif interval == "full":
            if hz == 254:
                self.classifier = nn.Linear(2840,4, bias = False)  
            elif hz == 1024:
                self.classifier = nn.Linear(12040,4, bias = False)
        
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm(res)
        res = self.elu(res)
        res = self.meanpool(res)
        res = self.dropout(res)
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res


In [ ]:
# network = ShallowCNN(hz = 254, interval = "action").float().to(device)
# accuracy_check(network, train_data, train_labels)

# print("####### TRAINING #######")
# op = optim.Adam(params = network.parameters(), lr = 0.001)
# lossf = nn.NLLLoss()
# train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 20, batch_size = 10, loss_func = lossf, optimizer = op)

# accuracy_check(network, train_data, train_labels)

ACCURACY: 0.2916666666666667
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.4583873748779297 	 1.3862940073013306
1 	  1.13673996925354 	 1.386293649673462
2 	  0.9523791472117106 	 1.3862934112548828
3 	  0.8504157861073812 	 1.3862941265106201
4 	  0.6840148766835531 	 1.386291742324829
5 	  0.584118922551473 	 1.3862934112548828
6 	  0.4808325370152791 	 1.386292815208435
7 	  0.3956715265909831 	 1.3863235712051392
8 	  0.30614809195200604 	 1.3863489627838135
9 	  0.25180919965108234 	 1.386505365371704
10 	  0.23350663979848227 	 1.3867957592010498
11 	  0.18432295322418213 	 1.3874318599700928
12 	  0.1538578470547994 	 1.3893449306488037
13 	  0.12493910392125447 	 1.3938555717468262
14 	  0.10261507829030354 	 1.4093067646026611
15 	  0.07903365294138591 	 1.442307710647583
16 	  0.07039961715539296 	 1.5124176740646362
17 	  0.061933308839797974 	 1.6500489711761475
18 	  0.05529499053955078 	 1.9111230373382568
19 	  0.0515565425157547 	 2.222453594207763

In [ ]:
# accuracy_check(network, train_data, train_labels)
# accuracy_check(network, val_data, val_labels)
# accuracy_check(network, test_data, test_labels)

ACCURACY: 1.0
ACCURACY: 0.275
ACCURACY: 0.325
